In [75]:
%load_ext autoreload
%autoreload 2

import itertools as it
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import numpy as np
import networkx as nx
import torch
from torch_geometric.data import Data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction 

Machine learning has recently emerged as a powerful tool for predicting properties of quantum many-body systems. Generative models can learn from measurements of a single quantum state to accurately reconstruct the state and predict local observables for many ground states of Hamiltonians. In this tutorial, we focus on Rydberg atom systems and propose the use of conditional generative models to simultaneously represent a family of states by learning shared structures of different quantum states from measurements.

Refs: 

[Predicting Properties of Quantum Systems with Conditional Generative Models](https://arxiv.org/abs/2211.16943)

[Transformer Quantum State: A Multi-Purpose Model for Quantum Many-Body Problems](http://arxiv.org/abs/2208.01758)

[Bloqade](https://queracomputing.github.io/Bloqade.jl/dev/)

## Rydberg Hamiltonian 

The Hamiltonain can be written as 
$$
H=\frac{\Omega}{2} \sum_i X_i-\Delta \sum_i N_i+\sum_{i<j} \frac{V_0}{\left|\vec{x}_i-\vec{x}_j\right|^6} N_i N_j \text {, }
$$
where $\Omega$ is the [Rabi frequency](https://en.wikipedia.org/wiki/Rabi_frequency#:~:text=The\%20Rabi\%20frequency\%20is\%20the,intensity),  $\Delta$ is the detuning of a laser, $V_0$ is the Rydberg interaction constant, and $\vec{x}_i$ is the position vector of the site $i . N_i=\left|r_i\right\rangle\left\langle r_i\right|=\left(1+Z_i\right) / 2$ is the occupation number operator at site $i$, where $Z_i$ and $X_i=\left|r_i\right\rangle\left\langle g_i|+| g_i\right\rangle\left\langle r_i\right|$ are pseudo-Pauli operators.

The Hamiltonain is stoquastic. Thus the quantum state has no phases and can be represented as a classical probability distribution. 

## Representation of the quantum state:
Decomposing the joint distribution into a product of conditional distributions in an autoregressive manner,
$$
p_{\theta, \phi}\left(a_1, \ldots, a_n \mid \mathbf{x}\right)=\prod_{i=1}^n p_\theta\left(a_i \mid a_{i-1}, \ldots, a_1, g_\phi(\mathbf{x})\right),
$$
where $\theta$ denotes the set of parameters of the generative model, and $\phi$ are the parameters of the embedding network. We incorporate in $\mathbf{x}$ different values such as, $\Omega, \Delta, \beta$ and the graph structure of the rydberg system. Since both the embedding $g_\phi$ and the generative model $p_\theta$ are parametrized as neural networks, they can be trained in an end-to-end manner.

# The Graph Encoder Decoder Transformer architecture

In this tutorial, we will explain the network architecture used in the `get_rydberg_graph_encoder_decoder` function, which creates a RydbergEncoderDecoder model. This model is designed to process graph-structured data using a combination of Graph Convolutional Networks (GCNs) and the classic Encoder-Decoder architecture as introduced in [Vaswani et al.](https://papers.nips.cc/paper_files/paper/2017/hash/3f5ee243547dee91fbd053c1c4a845aa-Abstract.html).

In [6]:
import copy
from typing import Tuple

import torch
from pytorch_lightning import LightningModule
from torch import Tensor, nn
from torch_geometric.nn import GATConv, GCNConv

from rydberggpt.models.graph_embedding.models import GraphEmbedding
from rydberggpt.models.rydberg_encoder_decoder import RydbergEncoderDecoder

from rydberggpt.models.transformer.layers import DecoderLayer, EncoderLayer
from rydberggpt.models.transformer.models import (
    Decoder,
    Encoder,
    EncoderDecoder,
    Generator,
)
from rydberggpt.models.transformer.modules import (
    PositionalEncoding,
    PositionwiseFeedForward,
)
from rydberggpt.utils import to_one_hot

## Main components 

The `RydbergEncoderDecoder` model consists of the following main components:

**Encoder:** The encoder processes the input graph data and generates a continuous representation. It consists of multiple `EncoderLayer` blocks, each containing a multi-head self-attention mechanism and a position-wise feed-forward network, followed by layer normalization and dropout.

**Decoder:** The decoder takes the continuous representation generated by the encoder and produces the output predictions. It is composed of multiple `DecoderLayer` blocks, each containing two multi-head attention mechanisms (self-attention and encoder-decoder attention) and a position-wise feed-forward network, followed by layer normalization and dropout.

**src_embed:** This component is responsible for transforming the input graph data into a continuous representation. It uses the `GraphEmbedding` class, which employs `GCNConv` layers (or other graph convolution layers, such as `GATConv`) to process the graph structure. The number of graph layers can be controlled with the `num_layers` parameter.

**tgt_embed:** This is a sequential model that first applies a linear transformation to the target input states and then adds positional encoding to provide information about the sequence order. The positional encoding is applied using the `PositionalEncoding` class.

**Generator:** The generator is a simple linear layer that maps the output of the decoder to the desired output dimension (in this case, 2). It is used for producing the final output predictions.

In the `get_rydberg_graph_encoder_decoder` function, the model is created using the provided configuration (config). This configuration contains information about the model's dimensions, number of layers, and other hyperparameters. After initializing the model, the weights of the parameters with more than one dimension are initialized using [Xavier uniform initialization](https://paperswithcode.com/method/xavier-initialization).

Overall, this network architecture combines the power of graph convolutional networks for processing graph-structured data with the sequence-to-sequence learning capabilities of the Encoder-Decoder architecture. This allows the model to effectively learn complex patterns in both the graph structure and the sequence data.

In [8]:
def get_rydberg_graph_encoder_decoder(config):
    c = copy.deepcopy
    attn = nn.MultiheadAttention(config.d_model, config.num_heads, batch_first=True)
    position = PositionalEncoding(config.d_model, config.dropout)
    ff = PositionwiseFeedForward(config.d_model, config.d_ff, config.dropout)

    model = RydbergEncoderDecoder(
        encoder=Encoder(
            EncoderLayer(config.d_model, c(attn), c(ff), config.dropout),
            config.num_blocks_encoder,
        ),
        decoder=Decoder(
            DecoderLayer(config.d_model, c(attn), c(attn), c(ff), config.dropout),
            config.num_blocks_decoder,
        ),
        src_embed=GraphEmbedding(
            graph_layer=GCNConv,  # GATConv
            in_node_dim=config.in_node_dim,
            d_hidden=config.graph_hidden_dim,
            d_model=config.d_model,
            num_layers=config.graph_num_layers,
            dropout=config.dropout,
        ),
        tgt_embed=nn.Sequential(
            nn.Linear(config.num_states, config.d_model), c(position)
        ),
        generator=Generator(config.d_model, 2),
        config=config,
    )

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return model

## Loss function

The dataset is composed of $N_H$ Hamiltonians and obtain $N_s$ measurement outcomes for each ground state leading to a training set $\mathcal{D}$  of size $N_HN_s$. The training objective is the average negative log-likelihood loss, 
$$
\min _{\theta, \phi} \mathcal{L}(\theta, \phi):=\frac{1}{N_H N_s} \sum_{(\vec{a}, \mathbf{x}) \in \mathcal{D}}-\log p_{\theta, \phi}\left(a_1, \ldots, a_n \mid \mathbf{x}\right)
$$
corresponding to maximizing the conditional likelihoods over the observed measurment outcomes. 

## Graph Embedding in Rydberg Atom Systems



In our approach, we leverage graph neural networks (GNNs) to process the underlying graph structure of Rydberg atom systems. In these systems, the graph nodes represent the Rydberg atoms, and each node is assigned a node_feature vector containing information about the Rabi frequency (Ω), detuning (Δ), and temperature (β). The Rydberg blockade radius, which determines the interaction strength between atoms, is encoded as edge attributes in the graph.

GNNs are powerful tools for learning representations of graph-structured data, capturing both local and global information within the graph. In our model, we employ graph convolutional layers, such as GCNConv, to learn meaningful embeddings of the input graph. These embeddings take into account both node features and edge attributes, enabling the model to learn complex relationships between atoms in the Rydberg system.

To understand the basics of graph neural networks and their applications, we recommend the following resources:

1. [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/): This article provides an accessible and visually appealing introduction to GNNs, covering their motivation, core concepts, and various architectures.

2. [Understanding Convolutions on Graphs](https://distill.pub/2021/understanding-gnns/): This article dives deeper into the inner workings of GNNs, specifically focusing on convolution operations on graphs. It provides insights into how graph convolutions can be understood as message-passing mechanisms and how they can be generalized.

3. [Pytorch_geometric](https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html): PyTorch Geometric is a library for deep learning on irregular input data such as graphs, point clouds, and manifolds. It provides efficient implementations of various GNN layers and models, making it easier to implement and experiment with graph-based neural networks. This resource serves as a guide to getting started with the library and provides documentation for its various features.

In our Rydberg atom system model, the graph embedding component serves as a crucial bridge between the graph-structured input data and the encoder-decoder architecture. By leveraging the capabilities of GNNs, we can effectively learn complex patterns in the graph structure and enhance the performance of our model for predicting properties of quantum many-body systems.